## Game interface

In [1]:
class Game:
    
    def __init__(self, player1, player2):
        self.gamestate = [[' ' for i in range(81)], -1, [' ' for i in range(9)]]
        self.turn = 0
        self.players = (player1, player2)
        self.history = []
        
        self.register_players()
        
    def register_players(self):
        for player in self.players:
            player.register_game(self)
    
    def get_legal_moves(self, gamestate = None):
        if not gamestate:
            gamestate = self.gamestate
        gameboard = gamestate[0]
        gamerange = gamestate[1]
        gamemacro = gamestate[2]
        if gamerange != -1 and gamemacro[gamerange] == ' ':
            legal_moves = [9*gamerange+pos for pos, cell in enumerate(gameboard[9*gamerange:9*gamerange+9]) if cell == ' ']
            if legal_moves:
                return legal_moves
        return [pos for pos, cell in enumerate(gameboard) if cell == ' ' and gamemacro[int(pos/9)] == ' ']
    
    def project_move(self, gamestate, move, turn):
        projected_gameboard = list(gamestate[0])
        projected_gameboard[move] = turn
        return projected_gameboard, move % 9, self.update_macro(projected_gameboard, list(gamestate[2]))
    
    def update_macro(self, gameboard, gamemacro):            
        lines = [[0,1,2], [3,4,5], [6,7,8], [0,3,6], [1,4,7], [2,5,8], [0,4,8], [6, 4, 2]]
        for index, macro in enumerate(gamemacro):
            if not macro == ' ':
                continue
            for line in lines:
                check = [1 for cell in line if gameboard[cell + index*9] == self.turn]
                if sum(check) == 3:
                    gamemacro[index] = self.turn
                    break
                check = [1 for cell in line if gameboard[cell + index*9] == int(not self.turn)]
                if sum(check) == 3:
                    gamemacro[index] = int(not self.turn)
                    break
        return gamemacro
    
    def make_move(self, move):
        self.gamestate[0][move] = self.turn
        self.gamestate[1] = move % 9
        self.gamestate[2] = self.update_macro(self.gamestate[0], self.gamestate[2])
        
        self.history.append(move)
        self.turn = int(not self.turn)
        
    def get_boardstate(self, gamestate = None):
        if not gamestate:
            gamestate = self.gamestate
        row_indexes = [0,1,2,9,10,11,18,19,20]
        offsets = [0,3,6,-1,27,30,33,-1,54,57,60]
        for offset in offsets:
            if offset == -1:
                print(21*'-')
                continue
            row = [str(gamestate[0][cell+offset]) for cell in row_indexes]
            row.insert(6,'|')
            row.insert(3,'|')
            print(' '.join(row))
        
        print('macro boardstate: ')
        print(gamestate[2][0], gamestate[2][1], gamestate[2][2])
        print(gamestate[2][3], gamestate[2][4], gamestate[2][5])
        print(gamestate[2][6], gamestate[2][7], gamestate[2][8])
    
    def result(self, gamestate = None):
        if not gamestate:
            gamestate = self.gamestate
        
        gameboard = gamestate[0]
        gamemacro = gamestate[2]
        
        lines = [[0,1,2], [3,4,5], [6,7,8], [0,3,6], [1,4,7], [2,5,8], [0,4,8], [6, 4, 2]]
        for line in lines:
            check = [gamemacro[cell]*2 - 1 for cell in line if gamemacro[cell] != ' ']
            if sum(check) == 3 or sum(check) == -3:
                player = (sum(check)+3)/6
                player = int(player)
                return 'player '+str(player)+' won'
        if ' ' not in gamemacro:
            return 'draw'
        if not self.get_legal_moves():
            enemy = int(not self.turn)
            return 'player '+str(enemy)+' won'
        return 'pending'
    
    def game_start(self, hide = False):        
        while self.result() == 'pending':
            self.players[self.turn].move()
        if not hide:
            self.get_boardstate()
        return self.result()

In [2]:
from random import choice

class RandomAlgorithm:

    def __init__(self):
        pass
    
    def register_game(self, game):
        self.game = game
    
    def move(self):
        all_moves = self.game.get_legal_moves()
        self.game.make_move(choice(all_moves))

class PlayerInterface:

    def __init__(self):
        pass

    def register_game(self, game):
        self.game = game
    
    def move(self):
        self.game.get_boardstate()
        print('all legal moves', self.game.get_legal_moves())
        move = int(input('enter move here: '))
        self.game.make_move(move)

In [3]:
# is this game fair?
from collections import Counter
counter = Counter()
for i in range(1000):
    game = Game(RandomAlgorithm(), RandomAlgorithm()) 
    simulated_result = game.game_start(True)
    counter.update([simulated_result])

print(counter)

Counter({'player 0 won': 527, 'player 1 won': 421, 'draw': 52})


### Minimax algorithm
No alpha-beta pruning implemented

In [4]:
class MinimaxAlgorithm:

    def __init__(self, depth):
        self.depth = depth
    
    def register_game(self, game):
        self.game = game
    
    def move(self):
        
        def evaluate(turn, gamestate, macro):
            lines = [[0,1,2], [3,4,5], [6,7,8], [0,3,6], [1,4,7], [2,5,8], [0,4,8], [6, 4, 2]]
            enemy = int(not turn)
            value = 0
            for line in lines:
                roi = [gamestate[0][macro*9+pos] for pos in line]
                if roi.count(turn) == 2 and enemy not in roi:
                    value += 20
                if roi.count(turn) == 3:
                    value += 50
                    break
                if roi.count(enemy) == 2 and turn not in roi:
                    value -= 40
                
                macro_roi = [gamestate[2][pos] for pos in line]
                if macro_roi.count(turn) == 2 and enemy not in macro_roi:
                    value += 100
                if macro_roi.count(turn) == 3:
                    value += 999
                    break
                if macro_roi.count(enemy) == 2 and turn not in macro_roi:
                    value -= 200
            return value
        
        def dfs(turn, gamestate, depth):
            if depth == 0:
                return 0, 0
            
            enemy = int(not turn)
            
            all_moves = self.game.get_legal_moves(gamestate)
            if len(all_moves) == 0:
                return 0, 0

            all_values = []
            for possible_move in all_moves:
                macro = gamestate[1]
                new_gamestate = self.game.project_move(gamestate, possible_move, turn)
                value = evaluate(turn, new_gamestate, macro)
                value -= dfs(enemy, new_gamestate, depth - 1)[0]
                all_values.append(value)

            return max(all_values), all_moves[all_values.index(max(all_values))]
        
        best_move = dfs(self.game.turn, self.game.gamestate, self.depth)
        self.game.make_move(best_move[1])

### Monte Carlo tree search

In [5]:
from math import sqrt, log
from copy import deepcopy

class MonteCarloAlgorithm:

    class Node:
        def __init__(self, parent, move = None, algorithm = None):
            self.parent = parent
            self.children = {}
            self.t = 0
            self.n = 0
            self.move = move

            if parent and not algorithm:
                self.algorithm = parent.algorithm
                self.turn = int(not parent.turn)
                return
            self.algorithm = algorithm
            self.turn = 1
        
        def get_gamestate(self):
            def upward_traversal(node):
                if node == node.algorithm.root:
                    return [[' ' for i in range(81)], -1, [' ' for i in range(9)]]
                return node.algorithm.game.project_move(upward_traversal(node.parent), node.move, node.turn)
            return upward_traversal(self)
        
        def expand(self):
            if self.children:
                return
            gamestate = self.get_gamestate()
            possible_moves = self.algorithm.game.get_legal_moves(gamestate)
            self.children = {move:MonteCarloAlgorithm.Node(self, move = move) for move in possible_moves}
        
        def select(self):            
            if self.n == 0:
                score = self.path()
                self.n += 1
                self.t += score
                return score
            
            self.expand()
            if not self.children:
                return 0

            def ucb_value(child):
                if child.n == 0:
                    return 999999
                if self.algorithm.game.turn == 0:
                    exploitation = (child.n+child.t) / 2 / child.n
                elif self.algorithm.game.turn == 1:
                    exploitation = (child.n-child.t) / 2 / child.n
                exploration = 1.41 * sqrt(log(self.algorithm.pointer.n) / child.n)
                return exploitation + exploration
            values = [ucb_value(child) for child in self.children.values()]
            selected_node_index = values.index(max(values))
            selected_node = list(self.children.values())[selected_node_index]
            score = selected_node.select()
            self.n += 1
            self.t += score
            return score
        
        def path(self):
            simulated_game = Game(MinimaxAlgorithm(3), MinimaxAlgorithm(3))
            simulated_game.turn = self.turn
            simulated_game.gamestate = list(deepcopy(self.get_gamestate()))
            result = simulated_game.game_start(True)

            if result == 'player 0 won':
                return 1
            if result == 'player 1 won':
                return -1
            return 0
        
        def check_states(self, depth, maxdepth=None):
            if not maxdepth:
                maxdepth = depth
            print((maxdepth-depth) * '   ', self.t, self.n, self.move)
            if depth == 0 or not self.children:
                return
            for child in self.children.values():
                child.check_states(depth - 1, maxdepth)
    
    
    def __init__(self, depth):
        self.root = MonteCarloAlgorithm.Node(None, algorithm = self)
        self.pointer = self.root
        self.depth = pow(9, depth)
        
    def register_game(self, game):
        self.game = game
        
        self.gamestate = game.gamestate
        self.pointer = self.root

    def update_history(self):
        self.pointer = self.root
        for move in self.game.history:
            self.pointer.expand()
            self.pointer = self.pointer.children[move]
        self.gamestate = self.game.gamestate
            
    def move(self):
        self.update_history()
        self.pointer.expand()
        for i in range(self.depth):
            self.pointer.select()
                
        possible_moves = [(child.t, child.n, child.move) for child in self.pointer.children.values()]
        possible_moves = sorted(possible_moves)
        if self.game.turn == 0:
            best_move = possible_moves[-1]
        elif self.game.turn == 1:
            best_move = possible_moves[0]
        self.game.make_move(best_move[-1])

In [ ]:
player1 = MonteCarloAlgorithm(3)
game = Game(PlayerInterface(), player1)   
game.game_start()